In [1]:
import selenium #셀레니움
import pandas as pd #csv를 읽고 dataframe을 사용하기 위한 pandas
from selenium import webdriver #브라우저를 띄우고 컨트롤하기 위한 webdriver
from selenium.webdriver.common.keys import Keys #브라우저에 키입력 용
from selenium.webdriver.common.by import By #webdriver를 이용해 태그를 찾기 위함
from selenium.webdriver.support.ui import WebDriverWait #Explicitly wait을 위함
from webdriver_manager.chrome import ChromeDriverManager #크롬에서 크롤링 진행 크롬 웹 드라이버 설치시 필요
from selenium.webdriver.support import expected_conditions as EC #브라우저에 특정 요소 상태 확인을 위해
from bs4 import BeautifulSoup #브라우저 태그를 가져오고 파싱하기 위함
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException,TimeoutException #예외처리를 위한 예외들
from time import sleep
import numpy as np
#크롬 드라이버 최신 버전 https://googlechromelabs.github.io/chrome-for-testing/

c:\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
df_all=pd.read_csv('법정동_20240201.csv')
df_all.dropna(subset=['읍면동명'],inplace=True)
df_all=df_all.reset_index().drop('index',axis=1)
df_all.fillna('',inplace=True)
df_all=df_all.assign(
    지역 = lambda x:x['시도명']+' ' + x['시군구명']+' ' +x['읍면동명']+' '
)
df_all=df_all[['지역','시도명','시군구명','읍면동명']]
display(df_all['지역'].nunique(),df_all) #전국 => 5066번 검색

5066

,지역,시도명,시군구명,읍면동명
0,서울특별시 종로구 청운동,서울특별시,종로구,청운동
1,서울특별시 종로구 신교동,서울특별시,종로구,신교동
2,서울특별시 종로구 궁정동,서울특별시,종로구,궁정동
3,서울특별시 종로구 효자동,서울특별시,종로구,효자동
4,서울특별시 종로구 창성동,서울특별시,종로구,창성동
...,...,...,...,...
20273,전북특별자치도 부안군 위도면,전북특별자치도,부안군,위도면
20274,전북특별자치도 부안군 위도면,전북특별자치도,부안군,위도면
20275,전북특별자치도 부안군 위도면,전북특별자치도,부안군,위도면
20276,전북특별자치도 부안군 위도면,전북특별자치도,부안군,위도면


In [3]:
#여러 지역 단위 검색어
def scrape(keyword, regions):
    store_name_lst = []
    category_lst = []
    phone_num_lst = []
    address_lst = []
    score_lst = []
    visitor_review_lst = []
    df=pd.DataFrame()
    def switch_left():
    ############## iframe으로 왼쪽 포커스 맞추기 ##############
        driver.switch_to.parent_frame()
        iframe = driver.find_element(By.XPATH,'//*[@id="searchIframe"]')
        driver.switch_to.frame(iframe)
        
    def switch_right():
    ############## iframe으로 오른쪽 포커스 맞추기 ##############
        driver.switch_to.parent_frame()
        #가게명 클릭 후 우측 프레임 로딩 대기
        WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.ID, "entryIframe"))
        )
        sleep(0.1) #혹시 몰라 추가시간

        iframe = driver.find_element(By.XPATH,'//*[@id="entryIframe"]')
        driver.switch_to.frame(iframe)

    options = webdriver.ChromeOptions()
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')
    options.add_argument('window-size=1380,900')
    driver = webdriver.Chrome(options=options)
    driver.implicitly_wait(time_to_wait=3)

    driver.get("https://map.naver.com/v5/") 

    try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "input_search"))
        )
    finally:
        pass

    for region in regions:
        full_keyword = region + keyword

        search_box = driver.find_element(By.CLASS_NAME, "input_search")
        search_box.clear()
        search_box.send_keys(full_keyword)
        search_box.send_keys(Keys.RETURN)

        driver.implicitly_wait(5)

        switch_left()

        try:
            no_result_message = driver.find_element(By.CLASS_NAME, 'FYvSc').text
            if no_result_message == '조건에 맞는 업체가 없습니다.':
                print(f'지역: {region}, 검색어: {keyword} - 검색 결과 없음')
                driver.switch_to.default_content()
                driver.find_element(By.CLASS_NAME, "btn_clear").click()
                continue  
        except NoSuchElementException:  
            driver.switch_to.default_content()
            pass

        loop = True
        page_no = 1

        while (loop):
            switch_left()

            scrollable_element = driver.find_element(By.CLASS_NAME, "Ryr1F")
            last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)

            while True:
                driver.execute_script("arguments[0].scrollTop += 1500;", scrollable_element)
                sleep(0.5)
                new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)
                if new_height == last_height:
                    break
                last_height = new_height

            current_page_no = driver.find_element(By.XPATH,'//a[contains(@class, "mBN2s qxokY")]').text

            elements = driver.find_elements(By.XPATH,'//*[@id = "_pcmap_list_scroll_container"]/ul/li')

            print(f'현재 {current_page_no} 페이지 / 총 {len(elements)}개(광고 포함)의 가게를 찾았습니다.\n')

            switch_left()

            for index, e in enumerate(elements, start=1):
                store_name = ''
                category = ''
                phone_num = ''
                address = ''

                switch_left()

                if e.get_attribute("data-laim-exp-id")=="undefined": 
                    e.find_elements(By.XPATH, "./div[1]/div")[-1].find_element(By.XPATH,"./a[1]/div/div/span[1]").click() 

                elif e.get_attribute("data-laim-exp-id")=="undefinedundefined":
                    e.find_element(By.CLASS_NAME,'CHC5F').find_element(By.XPATH, "./a/div/div/span[1]").click() 

                else:
                    continue 

                switch_right()

                title = driver.find_element(By.XPATH,'//div[@class = "zD5Nm undefined"]')

                store_name = title.find_element(By.XPATH,'.//div[1]/div[1]/span[1]').text
                category = title.find_elements(By.XPATH,'.//div[1]/div[1]/span')[-1].text

                try:
                    phone_num = driver.find_element(By.XPATH,'//span[@class = "xlx7Q"]').text
                    driver.implicitly_wait(0.5)
                except:
                    phone_num = '-'

                address = driver.find_element(By.XPATH,'//span[@class = "LDgIH"]').text

                try:
                    reviews = title.find_elements(By.XPATH, './/div[@class = "dAsGb"]/span[@class = "PXMot"]')
                    visitor_review =[review.find_element(By.XPATH,'./a').text for review in reviews]
                    driver.implicitly_wait(0.1)
                except:
                    visitor_review = []

                try:
                    score = title.find_element(By.XPATH, './/div[@class = "dAsGb"]/span[@class = "PXMot LXIwF"]').text
                    driver.implicitly_wait(0.1)
                except:
                    score = '-' 

                print(f'가게명: {store_name},   업종: {category},   전화번호: {phone_num},   주소: {address},   별점: {score}   리뷰: {visitor_review}')
                store_name_lst.append(store_name)
                category_lst.append(category)
                phone_num_lst.append(phone_num)
                address_lst.append(address)
                score_lst.append(score)
                visitor_review_lst.append(visitor_review)

            switch_left()

            next_page = driver.find_elements(By.XPATH,'//*[@id="app-root"]/div/div[2]/div[2]/a')[-1]

            if(next_page.get_attribute('aria-disabled') == 'false'):
                driver.find_elements(By.XPATH,'//*[@id="app-root"]/div/div[2]/div[2]/a')[-1].click()
            else:
                loop = False 

        driver.switch_to.default_content()
        driver.find_element(By.CLASS_NAME, "btn_clear").click()
        driver.implicitly_wait(5)

    driver.quit() 
    def uni(df):
        return str(df[3:])
    
    df = pd.DataFrame(zip(store_name_lst, category_lst, phone_num_lst, address_lst,score_lst,visitor_review_lst))
    df.columns = ['가게명', '업종', '전화번호', '주소','별점','리뷰 수']
    df=df.reset_index().drop('index',axis=1)
    df['별점']=df['별점'].apply(uni)
    #프랜차이즈여부
    df1=df[df['가게명'].str.endswith('점')]

    df1.loc[:,'가게명'] = df1.loc[:,'가게명'].str.rsplit(" ", n=1).str.get(0)

    #위 상호명이 5개 이상인 경우는 프랜차이즈
    store_counts = df1.groupby('가게명').size()
    excluded_stores = store_counts[store_counts >= 5].index
    df['프랜차이즈여부'] = np.where(df['가게명'].isin(excluded_stores), 'O', 'X')
    return df

In [4]:
#단일 검색어

def scrape_data(keywords):
    store_name_lst = []
    category_lst = []
    phone_num_lst = []
    address_lst = []
    score_lst =[]
    visitor_review_lst =[]
    df=pd.DataFrame()
    def switch_left():
    ############## iframe으로 왼쪽 포커스 맞추기 ##############
        driver.switch_to.parent_frame()
        iframe = driver.find_element(By.XPATH,'//*[@id="searchIframe"]')
        driver.switch_to.frame(iframe)
        
    def switch_right():
    ############## iframe으로 오른쪽 포커스 맞추기 ##############
        driver.switch_to.parent_frame()
        #가게명 클릭 후 우측 프레임 로딩 대기
        WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.ID, "entryIframe"))
        )
        sleep(0.1) #혹시 몰라 추가시간

        iframe = driver.find_element(By.XPATH,'//*[@id="entryIframe"]')
        driver.switch_to.frame(iframe)

    options = webdriver.ChromeOptions()
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')
    options.add_argument('window-size=1380,900')

    driver = webdriver.Chrome(options=options)
    driver.implicitly_wait(time_to_wait=3)

    driver.get("https://map.naver.com/v5/") 

    try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "input_search"))
        )
    finally:
        pass

    keyword = keywords
    search_box = driver.find_element(By.CLASS_NAME, "input_search")
    search_box.send_keys(keyword)
    search_box.send_keys(Keys.ENTER)

    driver.implicitly_wait(5)

    loop = True

    while(loop):
        switch_left()

        scrollable_element = driver.find_element(By.CLASS_NAME, "Ryr1F")
        last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)

        while True:
            driver.execute_script("arguments[0].scrollTop += 1500;", scrollable_element)
            sleep(0.5)
            new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)
            if new_height == last_height:
                break
            last_height = new_height

        page_no = driver.find_element(By.XPATH,'//a[contains(@class, "mBN2s qxokY")]').text
        elements = driver.find_elements(By.XPATH,'//*[@id = "_pcmap_list_scroll_container"]/ul/li')

        print('현재 ' + '\033[95m' + str(page_no) + '\033[0m' + ' 페이지 / '+ '총 ' + '\033[95m' + str(len(elements)) + '\033[0m' + '개(광고 포함)의 가게를 찾았습니다.\n')

        switch_left()

        for index, e in enumerate(elements, start=1):
            store_name = ''
            category = ''
            phone_num = ''
            address = ''

            switch_left()

            if e.get_attribute("data-laim-exp-id")=="undefined": 
                e.find_elements(By.XPATH, "./div[1]/div")[-1].find_element(By.XPATH,"./a[1]/div/div/span[1]").click() 

            elif e.get_attribute("data-laim-exp-id")=="undefinedundefined":
                e.find_element(By.CLASS_NAME,'CHC5F').find_element(By.XPATH, "./a/div/div/span[1]").click() 

            else:
                continue 

            switch_right()

            title = driver.find_element(By.XPATH,'//div[@class = "zD5Nm undefined"]')

            store_name = title.find_element(By.XPATH,'.//div[1]/div[1]/span[1]').text
            category = title.find_elements(By.XPATH,'.//div[1]/div[1]/span')[-1].text

            try:
                phone_num = driver.find_element(By.XPATH,'//span[@class = "xlx7Q"]').text
                driver.implicitly_wait(0.5)
            except:
                phone_num = '-'

            address = driver.find_element(By.XPATH,'//span[@class = "LDgIH"]').text

            try:
                reviews = title.find_elements(By.XPATH, './/div[@class = "dAsGb"]/span[@class = "PXMot"]')
                visitor_review =[review.find_element(By.XPATH,'./a').text for review in reviews]
                driver.implicitly_wait(0.1)
            except:
                visitor_review = []

            try:
                score = title.find_element(By.XPATH, './/div[@class = "dAsGb"]/span[@class = "PXMot LXIwF"]').text
                driver.implicitly_wait(0.1)
            except:
                score = '-' 

            print(f'가게명: {store_name},   업종: {category},   전화번호: {phone_num},   주소: {address},   별점: {score}   리뷰: {visitor_review}')
            store_name_lst.append(store_name)
            category_lst.append(category)
            phone_num_lst.append(phone_num)
            address_lst.append(address)
            score_lst.append(score)
            visitor_review_lst.append(visitor_review)
        
        switch_left()

        next_page = driver.find_elements(By.XPATH,'//*[@id="app-root"]/div/div[2]/div[2]/a')[-1]

        if(next_page.get_attribute('aria-disabled') == 'false'):
            driver.find_elements(By.XPATH,'//*[@id="app-root"]/div/div[2]/div[2]/a')[-1].click()
        else:
            loop = False 
    driver.quit()
     
    def uni(df):
        return str(df[3:])
    
    df = pd.DataFrame(zip(store_name_lst, category_lst, phone_num_lst, address_lst,score_lst,visitor_review_lst))
    df.columns = ['가게명', '업종', '전화번호', '주소','별점','리뷰 수']
    df=df.reset_index().drop('index',axis=1)
    df['별점']=df['별점'].apply(uni)
    #프랜차이즈여부
    df1=df[df['가게명'].str.endswith('점')]

    df1.loc[:,'가게명'] = df1.loc[:,'가게명'].str.rsplit(" ", n=1).str.get(0)

    #위 상호명이 5개 이상인 경우는 프랜차이즈
    store_counts = df1.groupby('가게명').size()
    excluded_stores = store_counts[store_counts >= 5].index
    df['프랜차이즈여부'] = np.where(df['가게명'].isin(excluded_stores), 'O', 'X')
    return df


In [5]:
answer = input('자유 키워드로 검색하시겠습니까? (0:자유 1:시도 2:시군구 3:읍면동 4:전국)')
if answer == "0":
    keyword = input('검색어를 입력하세요.')
    df=scrape_data(keyword)
    print(df)
elif answer == "1":
    city = input('원하는 시도명을 입력하세요.')
    keyword = input('검색어를 입력하세요.')
    regions = df_all.query('시도명 == @city')['지역'].unique().tolist()
    df=scrape(keyword, regions)
    print(df)
elif answer == "2":
    city = input('원하는 시도명을 입력하세요.')
    district = input('원하는 시군구명을 입력하세요.')
    keyword = input('검색어를 입력하세요".')
    regions = df_all.query('시도명 == @city and 시군구명 == @district')['지역'].unique().tolist()
    df=scrape(keyword, regions)
    print(df)
elif answer == "3":
    city = input('원하는 시도명을 입력하세요.')
    district = input('원하는 시군구명을 입력하세요.')
    town = input('원하는 읍면동명을 입력하세요.')
    keyword = input('검색어를 입력하세요.')
    regions = df_all.query('시도명 == @city and 시군구명 == @district and 읍면동명 == @town')['지역'].unique().tolist()
    df=scrape(keyword, regions)
    print(df)

elif answer == "4":
    keyword = input('검색어를 입력하세요.')
    regions = df_all['지역'].unique().tolist()
    df=scrape(keyword, regions)
    print(df)
else:
    print("잘못된 입력입니다. 다시 실행해주세요.")

현재 1 페이지 / 총 3개(광고 포함)의 가게를 찾았습니다.

가게명: 종로태권도아카데미,   업종: 무예,격투기,   전화번호: 02-722-2001,   주소: 서울 종로구 필운대로 116 신우빌딩 2층,   별점: -   리뷰: ['방문자리뷰 6', '블로그리뷰 2']
가게명: 용인대한국,   업종: 태권도,   전화번호: 02-722-9095,   주소: 서울 종로구 통일로 246-10 402,   별점: -   리뷰: ['방문자리뷰 11', '블로그리뷰 13']
가게명: 한국태권도체육관,   업종: 무예,격투기,   전화번호: 02-733-9095,   주소: 서울 종로구 무악동 45-18,   별점: -   리뷰: ['방문자리뷰 4']
         가게명      업종         전화번호                       주소 별점  \
0  종로태권도아카데미  무예,격투기  02-722-2001  서울 종로구 필운대로 116 신우빌딩 2층      
1      용인대한국     태권도  02-722-9095    서울 종로구 통일로 246-10 402      
2   한국태권도체육관  무예,격투기  02-733-9095         서울 종로구 무악동 45-18      

                   리뷰 수 프랜차이즈여부  
0    [방문자리뷰 6, 블로그리뷰 2]       X  
1  [방문자리뷰 11, 블로그리뷰 13]       X  
2             [방문자리뷰 4]       X  


In [6]:
df

,가게명,업종,전화번호,주소,별점,리뷰 수,프랜차이즈여부
0,종로태권도아카데미,"무예,격투기",02-722-2001,서울 종로구 필운대로 116 신우빌딩 2층,,"[방문자리뷰 6, 블로그리뷰 2]",X
1,용인대한국,태권도,02-722-9095,서울 종로구 통일로 246-10 402,,"[방문자리뷰 11, 블로그리뷰 13]",X
2,한국태권도체육관,"무예,격투기",02-733-9095,서울 종로구 무악동 45-18,,[방문자리뷰 4],X
